# Load Generator for Placing Orders

**Try Endpoint Responce**

In [14]:
import requests

/Users/viktoriiavlasenko/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [32]:
url = 'http://localhost:8080/api/v1/orders'
order = {
  "items": [
    { "productId": "B002PD61Y4", "quantity": 2 },
    { "productId": "B002SZEOLG", "quantity": 1 }
  ]
}

r = requests.post(url, json = order)

print(f"Order is placed with http status {r.status_code}")

Order is placed with http status 201


## Retrieve Products Ids from Inventory Database

**Load Custom MySQl Connector**

In [1]:
from scripts.connector import get_connector

In [2]:
cnx = get_connector()
cursor = cnx.cursor()

cursor.execute("select distinct(id) from product;")

product_ids = list()

for id in cursor.fetchall():
    product_ids.append(id[0])

cursor.close()
cnx.close()

print(f"Product ids sample: {product_ids[:5]}")
print(f"Total unique ids: {len(product_ids)}")

Product ids sample: ['B002PD61Y4', 'B002SZEOLG', 'B003B00484', 'B003L62T7W', 'B004IO5BMQ']
Total unique ids: 1351


In [3]:
product_ids[0]

'B002PD61Y4'

## Generate Orders Load

In [9]:
import random

In [10]:
BE_HOSTNAME = 'http://localhost:8080'

**Generate order**

In [11]:
def place_order():
    order_items = []
    for p_id in random.sample(product_ids, k=random.randint(1, 3)):
        order_items.append({
            "productId": p_id,
            "quantity": random.randint(1, 3)
        })

    payload = {"items": order_items}

    r = requests.post(f"{BE_HOSTNAME}/api/v1/orders", json=payload)
    return payload, r.status_code

In [30]:
order_items, order_status = place_order()

print(f"Order {order_items} is placed with http status {order_status}")

Order {'items': [{'productId': 'B01JOFKL0A', 'quantity': 2}]} is placed with http status 201


**Create Load**

In [4]:
from concurrent.futures import ThreadPoolExecutor

In [12]:
def run_load(concurrency, requests_count):
    with ThreadPoolExecutor(max_workers=concurrency) as executor:
        futures = [executor.submit(place_order) for _ in range(requests_count)]
        for f in futures:
            f.result()

In [16]:
run_load(concurrency=50, requests_count=50)